In [2]:
import numpy as np
import pandas as pd

## 10.1 GroupBy 메카닉

**NaN과 GroupBy**

In [41]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})

In [42]:
# value 하나를 nan으로 수정
temp = df.iloc[4, 2]
df.iloc[4, 2] = np.nan
df

,key1,key2,data1,data2
0,a,one,-2.055875,0.112531
1,a,two,1.856877,1.070280
2,b,one,0.515358,0.909491
3,b,two,0.943725,1.113740
4,a,one,NaN,-0.900667


In [43]:
df.groupby(['key1', 'key2']).mean()  # nan은 제외하고 집계

data1     data2
key1 key2                    
a    one  -2.055875 -0.394068
     two   1.856877  1.070280
b    one   0.515358  0.909491
     two   0.943725  1.113740

In [44]:
df.groupby(['key1', 'key2']).size()  # 개수 셀 때는 nan 값 포함

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [45]:
# key(그룹 색인) 하나를 nan으로 수정
df.iloc[4, 2] = temp
df.iloc[4, 1] = np.nan
df

,key1,key2,data1,data2
0,a,one,-2.055875,0.112531
1,a,two,1.856877,1.070280
2,b,one,0.515358,0.909491
3,b,two,0.943725,1.113740
4,a,NaN,0.228802,-0.900667


In [46]:
df.groupby(['key1', 'key2']).mean()  # key가 누락된 값은 집계에서 제외

data1     data2
key1 key2                    
a    one  -2.055875  0.112531
     two   1.856877  1.070280
b    one   0.515358  0.909491
     two   0.943725  1.113740

In [47]:
df.groupby(['key1', 'key2']).size()  # 개수 셀 때도 key가 누락되면 제외

key1  key2
a     one     1
      two     1
b     one     1
      two     1
dtype: int64

### 10.1.1 그룹 간 순회하기

In [48]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})

In [49]:
list(df.groupby('key1'))

[('a',
    key1 key2     data1     data2
  0    a  one  1.460484 -1.409559
  1    a  two  1.264796  1.669218
  4    a  one -0.895528  0.124050),
 ('b',
    key1 key2     data1     data2
  2    b  one  0.502679  0.458808
  3    b  two -0.885012 -1.746545)]

In [50]:
dict(list(df.groupby('key1')))

{'a':   key1 key2     data1     data2
 0    a  one  1.460484 -1.409559
 1    a  two  1.264796  1.669218
 4    a  one -0.895528  0.124050,
 'b':   key1 key2     data1     data2
 2    b  one  0.502679  0.458808
 3    b  two -0.885012 -1.746545}

In [54]:
df.groupby('key1')['data1'].mean()

key1
a    0.609918
b   -0.191167
Name: data1, dtype: float64

In [55]:
df.groupby('key1')[['data2']]

In [57]:
df['key1']

0    a
1    a
2    b
3    b
4    a
Name: key1, dtype: object

In [74]:
df2 = df.drop('data2', axis=1)

df2['data1'].groupby([df2['key1']]).mean()

key1
a    0.609918
b   -0.191167
Name: data1, dtype: float64

In [85]:
df.groupby(['key1', 'key2'])[['data1']].agg(['mean', 'std'])

data1          
               mean       std
key1 key2                    
a    one   0.282478  1.665952
     two   1.264796       NaN
b    one   0.502679       NaN
     two  -0.885012       NaN

In [95]:
df = pd.DataFrame({'A': 'a a b'.split(),
                   'B': [1,2,3],
                   'C': [4,6,5]})
g = df.groupby('A')

In [96]:
g.sum()

,B,C
A,,
a,3,10
b,3,5


In [94]:
g[['B', 'C']].apply(lambda x: x / x.sum())

,B,C
0,0.333333,0.4
1,0.666667,0.6
2,1.000000,1.0


**groupby filter**

In [10]:
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                          'foo', 'bar'],
                   'B' : [1, 2, 3, 4, 5, 6],
                   'C' : [2.0, 5., 8., 1., 2., 9.]})

grouped = df.groupby('A')
grouped.filter(lambda x: x['B'].mean() > 3.)

,A,B,C
1,bar,2,5.0
3,bar,4,1.0
5,bar,6,9.0


In [15]:
dff = pd.DataFrame({"A": np.arange(8), "B": list("aabbbbcc")})
dff.groupby("B").filter(lambda x: len(x) > 2)  # [참고] len(dataframe)은 dataframe의 세로길이(행개수)를 반환한다.

8

In [21]:
dff["C"] = np.arange(8)
dff.groupby("B").filter(lambda x: len(x["C"]) > 2)

,A,B,C
2,2,b,2
3,3,b,3
4,4,b,4
5,5,b,5


In [26]:
dff.groupby("B").head(2)

,A,B,C
0,0,a,0
1,1,a,1
2,2,b,2
3,3,b,3
6,6,c,6
7,7,c,7


In [27]:
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                          'foo', 'bar'],
                   'B' : ['one', 'one', 'two', 'three',
                          'two', 'two'],
                   'C' : [1, 5, 5, 2, 5, 5],
                   'D' : [2.0, 5., 8., 1., 2., 9.]})

grouped = df.groupby('A')

In [82]:
grouped.transform(lambda x: x.max() - x.min())

,C,D
0,4,6.0
1,3,8.0
2,4,6.0
3,3,8.0
4,4,6.0
5,3,8.0


In [83]:
grouped[['C', 'D']].apply(lambda x: x.max() - x.min())

,C,D
A,,
bar,3.0,8.0
foo,4.0,6.0


In [75]:
tsdf = pd.DataFrame(np.random.randn(10, 3), columns=["A", "B", "C"], index=pd.date_range("1/1/2000", periods=10))

tsdf.iloc[3:7] = np.nan
tsdf

,A,B,C
2000-01-01,0.402069,-0.195422,1.120132
2000-01-02,2.063457,-0.364196,1.700383
2000-01-03,-1.274706,0.706573,1.226444
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,-1.515105,-1.930438,-0.172535
2000-01-09,-1.539278,-0.065758,1.085976
2000-01-10,-1.451868,1.372026,0.287098


In [3]:
arrays = [['Falcon', 'Falcon', 'Parrot', 'Parrot'],

          ['Captive', 'Wild', 'Captive', 'Wild']]

index = pd.MultiIndex.from_arrays(arrays, names=('Animal', 'Type'))

df = pd.DataFrame({'Max Speed': [390., 350., 30., 20.]},

                  index=index)

df

Max Speed
Animal Type              
Falcon Captive      390.0
       Wild         350.0
Parrot Captive       30.0
       Wild          20.0

**MultiIndex DataFrame을 dictionary로 mapping하는 방법**

In [11]:
#1 각 level의 색인 값을 튜플로 묶어 전달한다.
df.groupby({('Falcon', 'Captive'): 'a', ('Falcon', 'Wild') : 'b', ('Parrot', 'Captive') : 'a', ('Parrot', 'Wild') : 'b'}).sum()

,Max Speed
a,420.0
b,370.0


In [12]:
#2 level인자와 해당 level의 색인만 대응시키는 딕셔너리를 전달한다.
df.groupby({'Falcon': 'a', 'Parrot': 'a'}, level=0).sum()

,Max Speed
Animal,
a,790.0
